<a href="https://colab.research.google.com/github/gitboku/statistics-practice/blob/master/%E3%83%AD%E3%82%B8%E3%82%B9%E3%83%86%E3%82%A3%E3%83%83%E3%82%AF%E5%9B%9E%E5%B8%B0%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ロジスティック回帰分析とは

目的変数を0から1の範囲でとりたいときに有効な分析方法。
例えば、癌の発生確率など。

## 非線形回帰とは

単回帰分析と重回帰分析は線警戒機。
非線形は直線ではない関係、例えば2上とかルートの関係。
ロジスティック回帰分析以外には多項式回帰分析とかサポートベクトル回帰分析などがある。

## 数式

ロジスティック回帰分析では以下の式を取る。

$$
p = \frac {1} {1+ \exp(-(a_1x_1 + \cdots + a_nx_n + b))}
$$

これを変形すると以下のようになる。

$$
\ln{\frac {p} {1-p}} = a_1x_1 + \cdots + a_nx_n + b
$$

子の式であれば重回帰分析と同じように最小二乗法で偏回帰係数を求めることができる。

---
また、上記の式をロジットと呼び、lで置くことがある。
このロジットに対して指数を取ると、オッズが出てくる。

$$
\exp(l) = \frac {p} {1 - p}
$$

このオッズはある事象が発生する確率と発生しない確率の比になっている。